# Developing the variant database

In [1]:
# reload modules before executing code in order to make development and debugging easier
%load_ext autoreload
%autoreload 2

In [2]:
# this jupyter notebook is running inside of the "notebooks" directory
# for relative paths to work properly, we need to set the current working directory to the root of the project
# for imports to work properly, we need to add the code folder to the system path
import os
from os.path import abspath, join, isdir, basename, isfile
import sys
if not isdir("notebooks"):
    # if there's a "notebooks" directory in the cwd, we've already set the cwd so no need to do it again
    os.chdir("..")
module_path = abspath("code")
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import sqlite3
import pandas as pd
import analysis as an

# Load HTCondor run data into pandas dataframes 

**TODO: Need a separate script to preprocess an HTCondor run before adding the results to the variant database.** The script could mimic a lot of what is done in condor_results.ipynb and should output the various plots and a text file with how many jobs are missing, etc.. This is needed for record keeping since the functions that load energies will ignore missing jobs. Also we just need to know exactly what we're putting in the database.

In [6]:
main_dir = "output/htcondor_runs/condor_energize_2021-03-31_15-29-09_gb1_ut3_1mv"
# condor log dir contains the condor .out, .err, and .log files for every job
condor_log_dir = join(main_dir, "output", "condor_logs")
# the energize out dir contains the output folder for every job
# each job's output folder has energies.csv and other output files
energize_out_dir = join(main_dir, "output", "energize_outputs")

In [7]:
# convenience functions in analysis.py allow us to load up HTCondor results very easily
energies_df, jobs_df, hparams_df = an.load_multi_job_results(energize_out_dir)

skipped 4 log directories because they did not contain all output files


In [8]:
display(energies_df.head())
display(jobs_df.head())
display(hparams_df.head())

,pdb_fn,variant,job_uuid,start_time,run_time,total_score,dslf_fa13,fa_atr,fa_dun,fa_elec,...,env,hs_pair,linear_chainbreak,overlap_chainbreak,pair,rg,rsigma,sheet,ss_pair,vdw
0,2qmt_p.pdb,"L12D,T51I,T55K",fT8dtQWjhUyL,2021-04-01 12:36:19,114,-215.580,0.0,-321.204,67.050,-142.686,...,-22.043,-4.155,0.0,0.0,-2.443,33.334,-22.923,0.343,-33.085,0.000
1,2qmt_p.pdb,"L12M,D22I,E56W",fT8dtQWjhUyL,2021-04-01 11:19:33,117,-201.816,0.0,-331.276,63.406,-121.336,...,-17.262,-3.540,0.0,0.0,2.034,33.474,-22.270,0.343,-31.007,0.000
2,2qmt_p.pdb,"Y3M,N35P,G41E",fT8dtQWjhUyL,2021-04-01 11:27:35,127,-195.776,0.0,-323.635,65.251,-128.688,...,-19.106,-2.687,0.0,0.0,1.944,33.799,-23.704,0.343,-34.502,0.000
3,2qmt_p.pdb,"L5H,A20I,T25D",fT8dtQWjhUyL,2021-04-01 11:49:10,117,-191.934,0.0,-329.467,75.982,-145.724,...,-20.848,-4.414,0.0,0.0,-1.401,33.533,-24.999,0.343,-34.136,0.193
4,2qmt_p.pdb,"K10S,K28L,T53W",fT8dtQWjhUyL,2021-04-01 12:45:31,113,-215.292,0.0,-329.860,65.334,-142.008,...,-12.984,-4.272,0.0,0.0,1.213,33.168,-22.714,0.343,-32.998,0.000


,uuid,cluster,process,hostname,github_commit_id,script_start_time
0,fT8dtQWjhUyL,14026430,9220,e1104.chtc.wisc.edu,v0.2,2021-04-01 09:51:21
1,MmhJ6hVPWRkn,14026430,6692,gpulab2000.chtc.wisc.edu,v0.2,2021-04-01 06:59:21
2,EZiQszkmCKsx,14026430,6147,e420.chtc.wisc.edu,v0.2,2021-04-01 06:35:38
3,nbQyYNxQ4fgD,14026430,9072,interactive2001.chtc.wisc.edu,v0.2,2021-04-01 09:35:56
4,dzz2eXbxeTpy,14026430,6728,e342.chtc.wisc.edu,v0.2,2021-04-01 07:06:56


,job_uuid,mutate_default_max_cycles,relax_distance,relax_repeats,relax_nstruct
0,fT8dtQWjhUyL,100.0,10.0,15.0,1.0
1,MmhJ6hVPWRkn,100.0,10.0,15.0,1.0
2,EZiQszkmCKsx,100.0,10.0,15.0,1.0
3,nbQyYNxQ4fgD,100.0,10.0,15.0,1.0
4,dzz2eXbxeTpy,100.0,10.0,15.0,1.0


# Add results to database
The database has multiple tables:
- variant
- job
- pdb_file

Every job will produce new records for *variant* and *job*, but not necessarily for *pdb_file*. The *pdb_file* table should be populated elsewhere, based on the prepared_pdb_files directory. With the PDB file, it should be easy to reference the correct foreign key when inserting new records into *variant*, as the *pdb_file* table uses the pdb filename as the primary key. 

In [20]:
db_fn = "variant_database/database.db"
con = sqlite3.connect(db_fn)

# energies go into the 'variant' table, but must rename "variant" column to "mutations"
# all other columns should match the default coming from load_energies
energies_db_ready = energies_df.rename(columns={"variant": "mutations"})
# if all or some of the records already exist, pandas/sqlite will throw an IntegrityError
# workaround is possible using a temp table to ignore existing records and just add any new ones
# https://stackoverflow.com/questions/30631848/insert-ignore-pandas-dataframe-into-mysql
# also, pandas is working on a new feature for "if_exists" that will allow to ignore existing records
# https://github.com/pandas-dev/pandas/pull/29636
# https://github.com/pandas-dev/pandas/issues/15988
# for now, just assume that if we get an integrity error then all records from this htcondor run were added
try:
    energies_db_ready.to_sql("variant", con, if_exists="append", index=False)
except sqlite3.IntegrityError:
    pass

con.close()

In [26]:
db_fn = "variant_database/database.db"
con = sqlite3.connect(db_fn)

# job info dataframe must be merged with the hparam dataframe as they are both in the same SQL table
# also add an "hp_" prefix to the hparams dataframe because that's what the SQL table expects
hparams_db_ready = hparams_df.add_prefix("hp_").rename(columns={"hp_job_uuid": "uuid"})
jobs_db_ready = pd.merge(jobs_df, hparams_db_ready, on="uuid")

# github_commit_id --> github_tag
jobs_db_ready = jobs_db_ready.rename(columns={"github_commit_id": "github_tag"})

try:
    jobs_db_ready.to_sql("job", con, if_exists="append", index=False)
except sqlite3.IntegrityError:
    pass

con.close()
        